In [ ]:
"""
Created on Wed Jan 15 09:28:16 2020
Group 7
@authors : G.H.
"""

In [25]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import numpy as np
import pandas as pd
import math
import operator
import os
from PIL import Image
from tqdm import tqdm_notebook as tqdm
import random
import pickle

import shutil
from shutil import copyfile

# keras : librairie de deep learning
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

# import for transfer learning
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [26]:
%run g7_functions_for_models.ipynb

# Lecure avec keras

In [27]:
df_seat_annot = pd.read_csv('../g7_SEATGURU_annotate.csv', sep=';')

In [28]:
# path
project_path = './../'
seatguru_path = 'All Data/ANALYSE IMAGE/IMG SEATGURU/'
hackathon_path = './../Inputs Hackathon/'
crea_path = project_path + 'G7_SEATGURU/Int/Airbus/'
new_paths = [crea_path + 'data_train', crea_path + 'data_test']
print(crea_path + 'data_train')

# Airbus type
airbus_planes = ['A320', 'A321', 'A330', 'A350', 'A380']

# SEATGURU
# Create a directory for each class
create_dirs_seatguru_type(df_seat_annot, project_path, seatguru_path, crea_path, aircraft_types=airbus_planes,
                          view = 'Int', man = 'Airbus')

# Split train and test
split_train_test_seatguru_type(new_paths=new_paths, path=crea_path, aircraft_types=airbus_planes)

# HACKATHON
# Creation of train and test samples for Hackathon pictures
split_train_test_hack(new_paths, hackathon_path, airbus_planes = ['A320', 'A330', 'A350', 'A380'], del_path = False)

./../G7_SEATGURU/Int/Airbus/data_train
./../G7_SEATGURU/Int/Airbus/A320
A320: 166 images
./../G7_SEATGURU/Int/Airbus/A321
A321: 121 images
./../G7_SEATGURU/Int/Airbus/A330
A330: 257 images
./../G7_SEATGURU/Int/Airbus/A350
A350: 39 images
./../G7_SEATGURU/Int/Airbus/A380
A380: 46 images


In [13]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies


train_generator = train_datagen.flow_from_directory(new_paths[0],
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_directory(new_paths[1],
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 1248 images belonging to 5 classes.
Found 529 images belonging to 5 classes.


# Transfer learning

In [14]:
# create the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# add a global spatial average pooling layer
x = base_model.output


x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

#x = MaxPooling2D(pool_size=(2, 2))(x)

# vector
x = Flatten()(x)

# let's add a fully-connected layer
x = Dense(1024)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(512)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(256)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)


# last layer which predict the class
predictions = Dense(5, activation = 'softmax')(x)

# this is the model we will trainll
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [16]:
step_size_train=train_generator.n//train_generator.batch_size

# Define learning rate reducer
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              patience=2, 
                              verbose=1, 
                              factor=0.5, 
                              min_lr=0.00001)

# How to include it in your training
model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=5,
                    validation_data=test_generator,
                    callbacks=[reduce_lr])  # here

Epoch 1/5
 1/39 [..............................] - ETA: 4:31 - loss: 2.0487 - accuracy: 0.1250

KeyboardInterrupt: 

In [15]:
path_mod = './../Models/'
save_model_classes(path_mod, 'Int_Airbus_Hackathon_Seatguru', train_generator, model)